+ 代码slide：https://courses.d2l.ai/zh-v2/assets/notebooks/chapter_convolutional-modern/batch-norm.slides.html#/
+ 视频地址： https://www.bilibili.com/video/BV1X44y1r77r?p=2

# 从0实现

+ 想实现batch_normalization这个层，首先要实现batch_norm这个操作，也就是那个公式计算
+ 类似当时实现卷积层的时候，先实现了二维互相关运算，再去实现卷积层

In [1]:
import torch
from torch import nn
import d2l_torch as d2l

实现batch_norm计算

+ github的issue：[Acquiring "is_grad_enabled()" inside an autograd function #56370](https://github.com/pytorch/pytorch/issues/56370)
+ pytorch文档：[TORCH.IS_GRAD_ENABLED](https://pytorch.org/docs/stable/generated/torch.is_grad_enabled.html?highlight=is_grad_enabled)
> Returns True if grad mode is currently enabled.   返回当前求导模式是否可用。

----
另外，搜索过程中发现更常见的一个语句是：`torch.set_grad_enabled(False)`
参考：[torch.set_grad_enabled(False)](https://blog.csdn.net/qq_40840797/article/details/119734575)
+ volatile是用来在验证或者测试的时候将输入设置成volatile，这样后继几点全部都被设置成volatile，也就是不需要自动求导。
+ 0.4.0以后采用torch.set_grad_enabled()来替代这种用法
+ 但是torch.set_grad_enabled()在使用的时候是设置一个上下文环境，也就是说只要设置了torch.set_grad_enabled(False)那么接下来所有的tensor运算产生的新的节点都是不可求导的，这个相当于一个全局的环境，即使是多个循环或者是在函数内设置的调用，只要torch.set_grad_enabled(False)出现，则不管是在下一个循环里还是在主函数中，都不再求导，除非单独设置一个孤立节点，并把他的requires_grad设置成true。


In [ ]:
def batch_norm(X,gamma,beta,moving_mean,moving_var,eps,momentum):
    """
    X:
    输入的数据（比如全连接层输出的，relu层之前）
    gamma和beta：
    就是可以学习的那两个超参数
    moving_mean和moving_var
    就是随机偏移和随机缩放，可以认为这是全局的均值和方差，是做推理的时候用的。
    可以认为是整个数据集上的均值和方差，而不是某个mini_batch上的均值和方差
    eps，
    就是为了防止方差为0，防止出现除0错误。如果不加这个简单的东西，可能一切都不同了
    一般也有个固定的值（轻易不要去修改）
    momentum
    是用来更新moving_mean和moving_var的一个东西，通常取0.9或者固定的一个东西
    
    """
    # 推理模式
    if not torch.is_grad_enabled(): # 如果梯度计算不可用（也就是推理模式）
        X_hat=(X-moving_mean)/torch.sqrt(moving_var+eps)  
    
    # 训练模式
    else:
        assert len(X.shape) in (2,4) 
        if len(X.shape)==2:  # 对于全连接层来说，作用在特征维
            mean=X.mean(dim=0) # 按照行求均值  全连接层的BN作用在特征维上  最终得到一个1xn的行向量， n表示全连接层输入数据的特征数
            var=((X-mean)**2).mean(dim=0)   # 方差=(每个样本-均值)的平方和，再除以样本数，看下面的公式。其实把求和和除以样本数这步，直接用mean方法代替了
        
        else: # 对于卷积层来说，作用在通道维度(也就是把所有通道对应位置加起来，求平均，得到的应该是 1x通道的高x通道的宽 这么一个矩阵 )
            mean=X.mean(dim=(0,2,3),keepdim=True) # 这里最终得到的就是 1xnx1x1的一个4d的结果，n表示输入/输出通道数
            var=((X-mean)**2.mean(dim=(0,2,3),keepdim=True))
           
        X-hat=(X-mean)/torch.sqrt(var+eps)
        6:19
            

验证全连接层的计算

推理：
+ 推理的时候，对输入数据的BN处理，其实是使用了全局的均值和方差
+ 因为推理的时候，可能X就只是一个样本，一个batch就是一个样本，那么这个时候如果还使用batch的话 就没法计算了，所以使用全局的均值和方差
+ 推理的时候，使用的全局的均值和方差其实来源于预测集的均值和方差（实际推理的时候，也默认新的数据和预测集分布一致，也可以使用预测集的均值和方差）

训练：
+ 首先判断是全连接层还是卷积层（只会作用在这两种层上，其它层不适用/没有写对应的处理函数），这里只设计了2d的卷积，其他1d和3d的不支持
    + 全连接层的输入维度是：batch_size，每个样本的维度（一维向量），比如：(32,64)表示每个样本是64维，每批是32个样本
    + 卷积层的输入维度是：(batch_size,输入通道数，高，宽)
+ 如果对axis或者dim有疑惑的话，可以去复习一下之前写的总结，[numpy中关于数组维度的理解——dim和axis](https://blog.csdn.net/Castlehe/article/details/116022827)

均值和方差计算公式：
+ 参考百度百科，点击[这里](https://baike.baidu.com/item/%E6%96%B9%E5%B7%AE%E8%AE%A1%E7%AE%97%E5%85%AC%E5%BC%8F/5318566?fr=aladdin)

平均值的计算公式为：
$$M=\frac{x_1+x_2+x_3+...+x_n}{n}$$

方差的计算公式为：
$$s^2=\frac{(x_1-M)^2+(x_2-M)^2+(x_3-M)^2+...+(x_n-M)^2}{n}$$

In [5]:
X=torch.rand((10,2))  # 创建10个样本，每个样本的特征数是2维
print(X)
mean=X.mean(dim=0)
print(mean)   #按照列求均值，最终得到各列特征的均值
print(((X-mean)**2).mean(dim=0))  
# (X-mean)**2 这个的维度其实和x一样，
# 所以后面的mean(dim=0)和求均值得到的结果维度也是一样的。  
# 都求的是某列特征的统计指标（均值/方差）

tensor([[0.2182, 0.9085],
        [0.3319, 0.3531],
        [0.3642, 0.6632],
        [0.3902, 0.5621],
        [0.3267, 0.0696],
        [0.1570, 0.3461],
        [0.4564, 1.0000],
        [0.5459, 0.1419],
        [0.0855, 0.2092],
        [0.5688, 0.2629]])
tensor([0.3445, 0.4517])
tensor([0.0224, 0.0924])


验证卷积层的计算

In [13]:
X=torch.rand((2,3,2,3)) # 两个三通道的，图像高为2，宽为3的输入
print(X)
mean=X.mean(dim=(0,2,3),keepdim=True)
print(mean,"\n",mean.shape)

tensor([[[[0.3377, 0.0040, 0.1068],
          [0.8988, 0.8476, 0.4406]],

         [[0.2373, 0.8555, 0.5407],
          [0.0875, 0.8951, 0.0701]],

         [[0.9436, 0.8168, 0.6496],
          [0.9063, 0.3542, 0.3073]]],


        [[[0.5269, 0.7035, 0.4117],
          [0.2906, 0.5506, 0.3359]],

         [[0.9111, 0.6993, 0.8829],
          [0.9526, 0.2769, 0.9301]],

         [[0.2550, 0.0682, 0.5667],
          [0.2577, 0.5471, 0.3435]]]])
tensor([[[[0.4546]],

         [[0.6116]],

         [[0.5013]]]]) 
 torch.Size([1, 3, 1, 1])


相当于先把所有batch加一起，然后把每个通道的高加起来，把通道的宽加起来，所有batch的所有通道最后就变成了 一个batch的通道数，比如上面的例子中，最终结果就是通道数个数字。 相当于对所有通道进行mean

逐步拆解一下，X.mean(dim=(0,2,3),keepdim=True)

In [11]:
dim_0_mean=X.mean(dim=0,keepdim=True)
print(f"dim=0的mean结果：\n{dim_0_mean}")  # 也可以把keepdim去掉看看 就是多一层括号而已，不影响什么


dim_0_2_mean=X.mean(dim=(0,2),keepdim=True)
dim_02_mean=dim_0_mean.mean(dim=(2),keepdim=True)
print(f"dim=(0,2)的mean结果：\n{dim_0_2_mean}")
print(f"对dim_0_mean进行dim=(2)的mean结果：\n{dim_02_mean}")  # 可以看到，结果是一样的，所以就是逐步对每个axis进行mean

dim=0的mean结果：
tensor([[[[0.3743, 0.6863, 0.5956],
          [0.7442, 0.5927, 0.2910]],

         [[0.5139, 0.9126, 0.3580],
          [0.8266, 0.9070, 0.7208]],

         [[0.7736, 0.8704, 0.1138],
          [0.3995, 0.3889, 0.5261]]]])
dim=(0,2)的mean结果：
tensor([[[[0.5593, 0.6395, 0.4433]],

         [[0.6703, 0.9098, 0.5394]],

         [[0.5865, 0.6297, 0.3200]]]])
对dim_0_mean进行dim=(2)的mean结果：
tensor([[[[0.5593, 0.6395, 0.4433]],

         [[0.6703, 0.9098, 0.5394]],

         [[0.5865, 0.6297, 0.3200]]]])


# 简洁实现